## general_import.py transposed to a .ipynb file for debugging

In [1]:
import numpy as np
import pandas as pd
import os
from DB2SalesforceAPI import DB2SalesforceAPI

sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4", 
}

In [2]:
cwd = os.getcwd()

In [3]:
#input new filename here
filename = 'MSI Faculty information - MSI EE Faculty.csv'#'Citrine Webinar Sent.csv'
f_type = filename.split('.')[-1]

In [4]:
#pandas import
if f_type == 'csv':
    #idf = pd.read_csv(filename)
    idf = pd.read_csv(filename)#,encoding="cp1252")
elif f_type == 'xlsx' or f_type == 'xls':
    xl = pd.ExcelFile(filename)
    print(xl.sheet_names)# see all sheet names
    sheet_names = xl.parse(xl.sheet_names) #this already performs an import
    
    idf = pd.read_excel(filename,sheet_name=0,header=0)

print(idf.head(2))

                             College Unnamed: 1      Faculty            Email  \
0  university of illinois at chicago    uic.edu  Pai-Yen Chen  pychen@uic.edu   
1  university of illinois at chicago    uic.edu  Mitra Dutta    dutta@uic.edu   

             Phone                    Title  \
0  (312) 996-2648       Associate Professor   
1  (312) 355-3221   Distinguished Professor   

                                          Department  \
0  Department of Electrical and Computer Engineering   
1  Department of Electrical and Computer Engineering   

                                           Interests  \
0  Optics and Optoelectronics, Dr. Chen’s group c...   
1  Optics and Optoelectronics, Optoelectronics de...   

  Klimeck-personal-acquaintance Product/Marketing Offering  ...  \
0                           NaN              Nanophotonics  ...   
1                            GK            Optoelectronics  ...   

             Faculty Homepage or personal webpage  \
0  https://ece.uic.ed

In [5]:
# selecting columns
try:
    idf = idf[['Email','Faculty','Product/Marketing Offering','Product/Marketing Offering 2']]
except:
    idf = idf[['Email','Faculty ','Product/Marketing Offering','Product/Marketing Offering 2']]
    idf = idf.rename(columns={'Faculty ':'Faculty'})
    
display(idf.head(2))

,Email,Faculty,Product/Marketing Offering,Product/Marketing Offering 2
0,pychen@uic.edu,Pai-Yen Chen,Nanophotonics,NaN
1,dutta@uic.edu,Mitra Dutta,Optoelectronics,NaN


In [6]:
#rename columns
idf = idf.rename(columns={'email':'Email','EMAIL':'Email'})
idf = idf.rename(columns={'Engagement Venue':'Venue__c'})
idf = idf.rename(columns={'First Name':'firstname','Last Name':'lastname'})
idf = idf.rename(columns={0:'Email'})
if name_flag == True:
    idf = idf.drop(columns='Name')#['NAME','LAST NAME','FIRST NAME'])

NameError: name 'name_flag' is not defined

In [7]:
display(idf.head(5))

,Email,Faculty,Product/Marketing Offering,Product/Marketing Offering 2
0,pychen@uic.edu,Pai-Yen Chen,Nanophotonics,NaN
1,dutta@uic.edu,Mitra Dutta,Optoelectronics,NaN
2,mazumder@uic.edu,Sudip K. Mazumder,Optoelectronics,ABACUS
3,lucyshi@uic.edu,J. Lucy Shi,Optoelectronics,ABACUS
4,stroscio@uic.edu,Michael Stroscio,Optoelectronics,ABACUS


In [8]:
idf = idf.dropna(subset=['Email'])
print(idf.shape)

(467, 4)


In [9]:
#email check rows
grows = []
brows = []
for ind,val in enumerate(idf['Email'].to_list()):
    if '@' in val:
        grows.append(ind)
    else:
        brows.append(ind)    
idf = idf.iloc[grows,:].reset_index().iloc[:,1:]
print(idf.head(2))

            Email       Faculty Product/Marketing Offering  \
0  pychen@uic.edu  Pai-Yen Chen              Nanophotonics   
1   dutta@uic.edu  Mitra Dutta             Optoelectronics   

  Product/Marketing Offering 2  
0                          NaN  
1                          NaN  


In [10]:
print(len(grows))
print(len(brows))

467
0


In [11]:
#firstname lastname split from faculty
faculty_raw = idf['Faculty'].to_list()
fnames = []
lnames = []

for ind,val in enumerate(faculty_raw):
    t_val = val.split(' ')
    if t_val[-1] == '':
        t_val = t_val[:-1]
    
    lnames.append(t_val[-1])
    fnames.append(' '.join(t_val[:-1]))
    

In [12]:
idf['firstname'] = fnames
idf['lastname'] = lnames
display(idf.head(5))

,Email,Faculty,Product/Marketing Offering,Product/Marketing Offering 2,firstname,lastname
0,pychen@uic.edu,Pai-Yen Chen,Nanophotonics,NaN,Pai-Yen,Chen
1,dutta@uic.edu,Mitra Dutta,Optoelectronics,NaN,Mitra,Dutta
2,mazumder@uic.edu,Sudip K. Mazumder,Optoelectronics,ABACUS,Sudip K.,Mazumder
3,lucyshi@uic.edu,J. Lucy Shi,Optoelectronics,ABACUS,J. Lucy,Shi
4,stroscio@uic.edu,Michael Stroscio,Optoelectronics,ABACUS,Michael,Stroscio


In [13]:
#check both in tandem
pm_offers1 = idf['Product/Marketing Offering'].to_list()
pm_offers2 = idf['Product/Marketing Offering 2'].to_list()
keep_ind = []

for ind,val in enumerate(pm_offers1):
    if val == 'ABACUS' or pm_offers2[ind] == 'ABACUS':
        keep_ind.append(ind)
    
print(keep_ind)

[2, 3, 4, 5, 6, 16, 17, 19, 20, 21, 22, 23, 29, 30, 32, 34, 35, 37, 38, 40, 42, 43, 47, 52, 59, 66, 77, 80, 82, 91, 92, 101, 123, 125, 129, 132, 133, 142, 146, 149, 152, 157, 158, 159, 160, 161, 164, 167, 171, 174, 179, 180, 181, 186, 187, 188, 189, 193, 194, 202, 203, 237, 243, 249, 253, 256, 257, 259, 260, 261, 262, 264, 266, 272, 298, 300, 309, 324, 325, 327, 337, 340, 341, 345, 348, 350, 354, 376, 387, 393, 411, 415, 417, 424, 426, 428, 429, 430, 443, 446, 447, 450, 451, 452, 453, 464, 465]


In [14]:
idf = idf.iloc[keep_ind,:]
display(idf.head(2))

,Email,Faculty,Product/Marketing Offering,Product/Marketing Offering 2,firstname,lastname
2,mazumder@uic.edu,Sudip K. Mazumder,Optoelectronics,ABACUS,Sudip K.,Mazumder
3,lucyshi@uic.edu,J. Lucy Shi,Optoelectronics,ABACUS,J. Lucy,Shi


In [15]:
idf = idf.reset_index()
idf = idf.drop(columns=['index','Faculty','Product/Marketing Offering','Product/Marketing Offering 2'])


In [16]:
idf['Venue__c'] = 'ABACUS'
display(idf.head(2))

,Email,firstname,lastname,Venue__c
0,mazumder@uic.edu,Sudip K.,Mazumder,ABACUS
1,lucyshi@uic.edu,J. Lucy,Shi,ABACUS


### Upsert Contacts with new Venue

In [17]:
import os
os_name = os.name
import platform
sys_name = platform.system() #Linux, Darwin, Windows

In [18]:
sys_name = 'Linux'

In [19]:
# salesforce queries for contact data
db_s = DB2SalesforceAPI(sf_login_params)

# deciding the queries
from copy import deepcopy
import_df_cols = deepcopy(idf.columns)
nh_id_flag = False
email_flag = False
if 'nanoHUB_user_ID__c' in import_df_cols:
    nh_id_flag = True

if 'Email' in import_df_cols:
    email_flag = True

Obtained Salesforce access token ...... True


In [20]:
if nh_id_flag == True and email_flag == True:
    sf_df = db_s.query_data('SELECT nanoHUB_user_ID__c, Email, Venue__c FROM Contact',sys_name=sys_name)
elif email_flag == True:
    sf_df = db_s.query_data('SELECT nanoHUB_user_ID__c, Email, Venue__c FROM Contact',sys_name=sys_name)

[Success] Bulk job creation successful. Job ID = 7505w00000ShIDLAA3
{"id":"7505w00000ShIDLAA3","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-12-10T16:09:36.000+0000","systemModstamp":"2020-12-10T16:09:36.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000ShIDLAA3","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-12-10T16:09:36.000+0000","systemModstamp":"2020-12-10T16:09:44.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000ShIDLAA3","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-12-10T16:09:36

In [21]:
# find all existing contacts
sf_emails = sf_df['Email'].to_list()
grows = []
brows = [] #dont need the sf_bad_rows as send to leads
sf_grows = []
for ind,val in enumerate(idf['Email'].to_list()):
    if val in sf_emails:
        grows.append(ind)
        sf_grows.append(sf_emails.index(val))
    else:
        brows.append(ind)

In [22]:
# pull the matching SF entries and the matching import df entries
sf_df_match = sf_df.iloc[sf_grows,:].reset_index().iloc[:,1:]
idf_match = idf.iloc[grows,:].reset_index().iloc[:,1:]
lead_df = idf.iloc[brows,:].reset_index().iloc[:,1:] #use this in next section

print(sf_df_match.head(2))
print(idf_match.head(2))

              Email Venue__c  nanoHUB_user_ID__c
0  stroscio@uic.edu      NaN             71464.0
1  skoester@umn.edu      NaN             48899.0
              Email firstname  lastname Venue__c
0  stroscio@uic.edu   Michael  Stroscio   ABACUS
1  skoester@umn.edu    Steven   Koester   ABACUS


In [23]:
# linear join since the sequence is matching
for ind,val in enumerate(sf_df_match['Venue__c']):
    try:
        val = val.split(';')
        val.append(idf['Venue__c'][ind])
        val = ';'.join(val)
    except:
        val = idf['Venue__c'][ind]
    sf_df_match['Venue__c'][ind] = val

print(sf_df_match.head(5))

              Email Venue__c  nanoHUB_user_ID__c
0  stroscio@uic.edu   ABACUS             71464.0
1  skoester@umn.edu   ABACUS             48899.0
2      tlow@umn.edu   ABACUS            292578.0
3   stadler@umn.edu   ABACUS             76666.0
4  sswisher@umn.edu   ABACUS            166194.0


C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [24]:
## delete duplicates
venues = sf_df_match['Venue__c'].to_list()
for ind,val in enumerate(venues):
    venues[ind]=';'.join(list(dict.fromkeys(val.split(';'))))
sf_df_match['Venue__c'] = venues
display(sf_df_match.head(5))

,Email,Venue__c,nanoHUB_user_ID__c
0,stroscio@uic.edu,ABACUS,71464.0
1,skoester@umn.edu,ABACUS,48899.0
2,tlow@umn.edu,ABACUS,292578.0
3,stadler@umn.edu,ABACUS,76666.0
4,sswisher@umn.edu,ABACUS,166194.0


In [25]:
## encoding correction for dashes
venues = sf_df_match['Venue__c'].apply(lambda x: x.replace('â\x80\x93','-'))
sf_df_match['Venue__c'] = venues

In [26]:
sf_df_match = sf_df_match.drop_duplicates()
display(sf_df_match.head(5))

,Email,Venue__c,nanoHUB_user_ID__c
0,stroscio@uic.edu,ABACUS,71464.0
1,skoester@umn.edu,ABACUS,48899.0
2,tlow@umn.edu,ABACUS,292578.0
3,stadler@umn.edu,ABACUS,76666.0
4,sswisher@umn.edu,ABACUS,166194.0


In [33]:
display(sf_df_match)

,Email,Venue__c,nanoHUB_user_ID__c
0,stroscio@uic.edu,ABACUS,71464.0
1,skoester@umn.edu,ABACUS,48899.0
2,tlow@umn.edu,ABACUS,292578.0
3,stadler@umn.edu,ABACUS,76666.0
4,sswisher@umn.edu,ABACUS,166194.0
5,danielkm@umd.edu,MSE SWS 2020 - A;Citrine Material Science Work...,296451.0
6,ltrombetta@uh.edu,ABACUS,11503.0
7,sergey.a.nikishin@ttu.edu,ABACUS,30301.0
8,drhodges@utep.edu,ABACUS,52294.0
9,dzubia@utep.edu,ABACUS,22314.0


In [27]:
print(sf_df_match.shape)

(26, 3)


## Begin upsert to SF

In [37]:
# rebuild api object
db_s_c = DB2SalesforceAPI(sf_login_params)

# send data to SF
db_s_c.object_id = 'Contact'
db_s_c.external_id = 'nanoHUB_user_ID__c'

db_s_c.send_data(sf_df_match)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000ShI38AAF
[Success] CSV upload successful. Job ID = 7505w00000ShI38AAF
[Success] Closing job successful. Job ID = 7505w00000ShI38AAF


In [38]:
# check status
db_s_c.check_bulk_status()

{'id': '7505w00000ShI38AAF',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-12-10T16:18:43.000+0000',
 'systemModstamp': '2020-12-10T16:18:45.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [39]:
from pprint import pprint

pprint(db_s_c.check_bulk_failed_results())

''


### Upsert Leads with new entries

In [40]:
#pull all current leads
db_s = DB2SalesforceAPI(sf_login_params)
sf_df = db_s.query_data('SELECT Email, Venue__c, SF_indexer__c FROM Lead')

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000ShIT9AAN
{"id":"7505w00000ShIT9AAN","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2020-12-10T16:18:54.000+0000","systemModstamp":"2020-12-10T16:18:54.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000ShIT9AAN","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2020-12-10T16:18:54.000+0000","systemModstamp":"2020-12-10T16:18:59.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6979,"retries":0,"totalProcessingTime":1047}
[Success] Bulk job completed successfully.


In [41]:
# find the max sf_indexer
indexers = sf_df['SF_indexer__c'].fillna(0).to_list()
max_ind = max(indexers)

In [42]:
# find all existing leads
sf_emails = sf_df['Email'].to_list()
m_rows = []
nm_rows = [] #don't need sf no match rows
sf_mrows = []

In [43]:
for ind,val in enumerate(lead_df['Email'].to_list()):
    if val in sf_emails:
        m_rows.append(ind)
        sf_mrows.append(sf_emails.index(val))
    else:
        nm_rows.append(ind)

In [44]:
# filter the matches
sf_df_match = sf_df.iloc[sf_mrows,:].reset_index().iloc[:,1:]
join_idf = lead_df.iloc[m_rows,:].reset_index().iloc[:,1:]
new_idf = lead_df.iloc[nm_rows,:].reset_index().iloc[:,1:]

In [45]:
print(sf_df_match.head(2))
print(join_idf.head(2))

             Email  SF_indexer__c Venue__c
0  lucyshi@uic.edu         1023.0      NaN
1     jcho@umn.edu          931.0      NaN
             Email   firstname lastname Venue__c
0  lucyshi@uic.edu     J. Lucy      Shi   ABACUS
1     jcho@umn.edu  Jeong-Hyun      Cho   ABACUS


In [46]:
# linear join since the sequence is matching
for ind,val in enumerate(sf_df_match['Venue__c']):
    try:
        val = val.split(';')
        #if 'MSE Summer Webinar Series 2020' in val:
        #    val.remove('MSE Summer Webinar Series 2020')
        #    if 'MSE Summer Webinar Series 2020' in val:
        #        val.remove('MSE Summer Webinar Series 2020')
        val.append(join_idf['Venue__c'][ind])
        val = ';'.join(val)
    except:
        val = join_idf['Venue__c'][ind]
    sf_df_match['Venue__c'][ind] = val    

print(sf_df_match.head(5))

              Email  SF_indexer__c                 Venue__c
0   lucyshi@uic.edu         1023.0                   ABACUS
1      jcho@umn.edu          931.0                   ABACUS
2  higma001@umn.edu         1089.0                   ABACUS
3  vlassovy@fiu.edu         2472.0  MSE SWS 2020 - S;ABACUS
4      neil@umd.edu         2491.0  MSE SWS 2020 - S;ABACUS


C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [47]:
## delete duplicates
venues = sf_df_match['Venue__c'].to_list()
for ind,val in enumerate(venues):
    venues[ind] = ';'.join(list(dict.fromkeys(val.split(';'))))
sf_df_match['Venue__c'] = venues
display(sf_df_match.head(5))

,Email,SF_indexer__c,Venue__c
0,lucyshi@uic.edu,1023.0,ABACUS
1,jcho@umn.edu,931.0,ABACUS
2,higma001@umn.edu,1089.0,ABACUS
3,vlassovy@fiu.edu,2472.0,MSE SWS 2020 - S;ABACUS
4,neil@umd.edu,2491.0,MSE SWS 2020 - S;ABACUS


In [48]:
# assign new SF_indexers for the new leads
new_max_ind = int(max_ind+new_idf.shape[0])
# print(new_idf.shape)
# print(max_ind)
# print(new_max_ind)
new_idf['SF_indexer__c'] = range(int(max_ind)+1,new_max_ind+1)

In [49]:
# need non-empty company field
new_idf['Company'] = '-'

In [50]:
new_idf.head(5)

,Email,firstname,lastname,Venue__c,SF_indexer__c,Company
0,mazumder@uic.edu,Sudip K.,Mazumder,ABACUS,2513,-
1,cclee@uci.edu,Chin,Lee,ABACUS,2514,-
2,hplee@uci.edu,Henry,Lee,ABACUS,2515,-
3,ramirtha@ucdavis.edu,Rajeevan,Amirtharajah,ABACUS,2516,-
4,rick.branner@gmail.com,G. Rick,Branner,ABACUS,2517,-


In [51]:
# ensure 'â\x80\x93' has been replaced
sf_df_match['Venue__c'] = sf_df_match['Venue__c'].apply(lambda x: x.replace('â\x80\x93','-'))
new_idf['Venue__c'] = new_idf['Venue__c'].apply(lambda x: x.replace('â\x80\x93','-'))

display(sf_df_match.head(20))
display(new_idf.head(20))

,Email,SF_indexer__c,Venue__c
0,lucyshi@uic.edu,1023.0,ABACUS
1,jcho@umn.edu,931.0,ABACUS
2,higma001@umn.edu,1089.0,ABACUS
3,vlassovy@fiu.edu,2472.0,MSE SWS 2020 - S;ABACUS
4,neil@umd.edu,2491.0,MSE SWS 2020 - S;ABACUS
5,agis@umd.edu,2488.0,MSE SWS 2020 - S;ABACUS
6,dcheng@fullerton.edu,1123.0,ABACUS
7,roger.lake@ucr.edu,2061.0,ABACUS
8,jianlin.liu@ucr.edu,1906.0,ABACUS
9,mihri.ozkan@ucr.edu,2047.0,ABACUS


,Email,firstname,lastname,Venue__c,SF_indexer__c,Company
0,mazumder@uic.edu,Sudip K.,Mazumder,ABACUS,2513,-
1,cclee@uci.edu,Chin,Lee,ABACUS,2514,-
2,hplee@uci.edu,Henry,Lee,ABACUS,2515,-
3,ramirtha@ucdavis.edu,Rajeevan,Amirtharajah,ABACUS,2516,-
4,rick.branner@gmail.com,G. Rick,Branner,ABACUS,2517,-
5,jsgomez@ucdavis.edu,J. Sebastian,Gomez-Diaz,ABACUS,2518,-
6,jhihath@ucdavis.edu,Joshua,Hihath,ABACUS,2519,-
7,hunt@ece.ucdavis.edu,Charles,Hunt,ABACUS,2520,-
8,sislam@ucdavis.edu,M. Saif,Islam,ABACUS,2521,-
9,bhkolner@ucdavis.edu,Brian,Kolner,ABACUS,2522,-


In [52]:
#drop duplicate rows
sf_df_match = sf_df_match.drop_duplicates()
new_idf = new_idf.drop_duplicates()

In [53]:
#send the matching ones
db_s_l1 = DB2SalesforceAPI(sf_login_params)

# send data to SF
db_s_l1.object_id = 'Lead'
db_s_l1.external_id = 'SF_indexer__c'

db_s_l1.send_data(sf_df_match)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000ShITiAAN
[Success] CSV upload successful. Job ID = 7505w00000ShITiAAN
[Success] Closing job successful. Job ID = 7505w00000ShITiAAN


In [59]:
# check status
db_s_l1.check_bulk_status()

{'id': '7505w00000ShITiAAN',
 'operation': 'upsert',
 'object': 'Lead',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-12-10T16:19:50.000+0000',
 'systemModstamp': '2020-12-10T16:19:52.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'SF_indexer__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 15,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 220,
 'apiActiveProcessingTime': 158,
 'apexProcessingTime': 26}

In [60]:
from pprint import pprint

pprint(db_s_l1.check_bulk_failed_results())

'"sf__Id","sf__Error",Email,SF_indexer__c,Venue__c\n'


In [56]:
#send the new ones
db_s_l2 = DB2SalesforceAPI(sf_login_params)

# send data to SF
db_s_l2.object_id = 'Lead'
db_s_l2.external_id = 'SF_indexer__c'

db_s_l2.send_data(new_idf)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000ShIU2AAN
[Success] CSV upload successful. Job ID = 7505w00000ShIU2AAN
[Success] Closing job successful. Job ID = 7505w00000ShIU2AAN


In [61]:
# check status
db_s_l2.check_bulk_status()

{'id': '7505w00000ShIU2AAN',
 'operation': 'upsert',
 'object': 'Lead',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-12-10T16:19:55.000+0000',
 'systemModstamp': '2020-12-10T16:19:58.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'SF_indexer__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 66,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 755,
 'apiActiveProcessingTime': 653,
 'apexProcessingTime': 55}

In [62]:
from pprint import pprint

pprint(db_s_l2.check_bulk_failed_results())

'"sf__Id","sf__Error",Company,Email,SF_indexer__c,Venue__c,firstname,lastname\n'
